In [218]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import loguniform  # Para distribuciones logarítmicas
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import loguniform  # Para distribuciones logarítmicas
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import loguniform, uniform
import numpy as np

### Optimizacion de modelos

| Modelo            | Ajuste de Hiperparámetros | Razón                                                                 |
|-------------------|---------------------------|-----------------------------------------------------------------------|
| **Naïve Bayes**   | **Grid Search**           | Pocos hiperparámetros (ej. `alpha` en `MultinomialNB`). Grid Search es rápido y suficiente para espacios pequeños. |
| **Regresión Logística** | **Grid Search** o **Random Search** | Grid Search si es simple (ej. probar `C` y `penalty`). Random Search si se añaden más parámetros (ej. `solver`, `tol`). |
| **SVM**           | **Random Search**         | Modelo complejo con espacio de búsqueda grande (ej. `C`, `kernel`, `gamma`). Random Search es más eficiente computacionalmente. |

## Naive Bayes optimizado con GridSearch

In [224]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report

# Definir el modelo base
nb_model = MultinomialNB()

# Definir los hiperparámetros a probar
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],  # Probar más valores de alpha
    'fit_prior': [True, False]  # Probar con y sin ajuste de prior
}

# Usar la métrica F1-macro para mejorar rendimiento en clases desbalanceadas
scorer = make_scorer(f1_score, average='macro')

# Configurar GridSearchCV
grid_search = GridSearchCV(nb_model, param_grid, cv=5, scoring=scorer, n_jobs=-1)

# Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train_tfidf, y_train)

# Obtener el mejor modelo y hacer predicciones
best_nb_model = grid_search.best_estimator_
y_pred_nb = best_nb_model.predict(X_test_tfidf)

# Evaluar el modelo optimizado
print("Mejor parámetro encontrado:", grid_search.best_params_)
print("Naïve Bayes Optimizado - Precisión en Test:")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))


Mejor parámetro encontrado: {'alpha': 0.1, 'fit_prior': False}
Naïve Bayes Optimizado - Precisión en Test:
Accuracy: 0.6101083032490975
              precision    recall  f1-score   support

           1       0.61      0.47      0.53       135
           2       0.27      0.14      0.18        93
           3       0.27      0.27      0.27       165
           4       0.35      0.42      0.38       311
           5       0.79      0.80      0.79       958

    accuracy                           0.61      1662
   macro avg       0.46      0.42      0.43      1662
weighted avg       0.61      0.61      0.61      1662



In [225]:
# 2. Preparar todos los componentes para guardar
model_components = {
    'model': best_nb_model,  # El modelo optimizado
    'grid_search': grid_search,  # Objeto completo de GridSearchCV
    'vectorizer': vectorizer,  # El vectorizador TF-IDF
    'label_encoder': le,  # El LabelEncoder
    'metadata': {
        'model_type': 'Optimized Naïve Bayes (GridSearchCV)',
        'best_parameters': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_accuracy': accuracy_score(y_test, y_pred_nb),
        'classification_report': classification_report(y_test, y_pred_nb, output_dict=True),
        'training_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'features': f"{X_train_tfidf.shape[1]} features TF-IDF",
        'classes': list(le.classes_),
        'cv_results': grid_search.cv_results_  # Todos los resultados de CV
    }
}

# 3. Guardar todo en un archivo .pkl
filename = f"optimized_naive_bayes_{datetime.datetime.now().strftime('%Y%m%d')}.pkl"
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"✅ Modelo Naïve Bayes optimizado guardado exitosamente en:\n{full_path}")
print(f"\n🔹 Mejores parámetros: {grid_search.best_params_}")
print(f"🔹 Mejor score (CV): {grid_search.best_score_:.4f}")
print(f"🔹 Accuracy (test): {accuracy_score(y_test, y_pred_nb):.4f}")

✅ Modelo Naïve Bayes optimizado guardado exitosamente en:
C:\Users\Vicky\Documents\ML_Amazon_Reviews_Analysis\models\optimized_naive_bayes_20250408.pkl

🔹 Mejores parámetros: {'alpha': 0.1, 'fit_prior': False}
🔹 Mejor score (CV): 0.4301
🔹 Accuracy (test): 0.6101


### Regresion Logistica optimizada con Grid Search

In [227]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Definir el modelo base
lr_model = LogisticRegression(max_iter=500)

# Definir los hiperparámetros a probar
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Control de regularización
    'penalty': ['l1', 'l2'],  # Tipos de regularización
    'solver': ['liblinear', 'saga']  # Solvers compatibles con l1 y l2
}

# Configurar GridSearchCV
grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train_tfidf, y_train)

# Obtener el mejor modelo y hacer predicciones
best_lr_model_grid = grid_search.best_estimator_
y_pred_lr_grid = best_lr_model_grid.predict(X_test_tfidf)

# Evaluar el modelo optimizado
print("🔹 Mejor parámetro encontrado (Grid Search):", grid_search.best_params_)
print("🔹 Regresión Logística Optimizada (Grid Search) - Precisión en Test:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr_grid))
print(classification_report(y_test, y_pred_lr_grid))


🔹 Mejor parámetro encontrado (Grid Search): {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
🔹 Regresión Logística Optimizada (Grid Search) - Precisión en Test:
Accuracy: 0.6660649819494585
              precision    recall  f1-score   support

           1       0.69      0.52      0.59       135
           2       0.38      0.10      0.15        93
           3       0.42      0.22      0.29       165
           4       0.48      0.25      0.33       311
           5       0.71      0.95      0.81       958

    accuracy                           0.67      1662
   macro avg       0.53      0.41      0.44      1662
weighted avg       0.62      0.67      0.62      1662



### Regresion logistica optimizada con Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Definir el modelo base
lr_model = LogisticRegression(max_iter=500)

# Definir los hiperparámetros con rangos más amplios
param_dist = {
    'C': np.logspace(-3, 3, 10),  # Valores en escala logarítmica
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(lr_model, param_distributions=param_dist, 
                                   n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

# Ajustar el modelo con los datos de entrenamiento
random_search.fit(X_train_tfidf, y_train)

# Obtener el mejor modelo y hacer predicciones
best_lr_model_random = random_search.best_estimator_
y_pred_lr_random = best_lr_model_random.predict(X_test_tfidf)

# Evaluar el modelo optimizado
print("🔹 Mejor parámetro encontrado (Random Search):", random_search.best_params_)
print("🔹 Regresión Logística Optimizada (Random Search) - Precisión en Test:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr_random))
print(classification_report(y_test, y_pred_lr_random))


In [ ]:
model_components = {
    'model': best_lr_model_grid,  # El modelo optimizado
    'grid_search': grid_search,  # Objeto completo de GridSearchCV
    'vectorizer': vectorizer,  # El vectorizador TF-IDF
    'label_encoder': le,  # El LabelEncoder
    'metadata': {
        'model_type': 'Optimized Logistic Regression (GridSearchCV)',
        'best_parameters': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_accuracy': accuracy_score(y_test, y_pred_lr_grid),
        'classification_report': classification_report(y_test, y_pred_lr_grid, output_dict=True),
        'training_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'features': f"{X_train_tfidf.shape[1]} features TF-IDF",
        'classes': list(le.classes_),
        'cv_results': grid_search.cv_results_  # Todos los resultados de CV
    }
}

# 3. Guardar todo en un archivo .pkl
filename = f"optimized_logistic_regression_grid_{datetime.datetime.now().strftime('%Y%m%d')}.pkl"
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"✅ Modelo de Regresión Logística optimizado (GridSearch) guardado exitosamente en:\n{full_path}")
print(f"\n🔹 Mejores parámetros: {grid_search.best_params_}")
print(f"🔹 Mejor score (CV): {grid_search.best_score_:.4f}")
print(f"🔹 Accuracy (test): {accuracy_score(y_test, y_pred_lr_grid):.4f}")

### SVM optimizado con Random Search

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Definir el modelo base
svm_model = SVC()

# Definir los hiperparámetros con rangos amplios
param_dist = {
    'C': np.logspace(-3, 3, 10),  # Valores en escala logarítmica (0.001 a 1000)
    'kernel': ['linear', 'rbf', 'poly'],  # Probamos diferentes kernels
    'gamma': ['scale', 'auto']  # Opciones estándar para gamma
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(svm_model, param_distributions=param_dist, 
                                   n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

# Ajustar el modelo con los datos de entrenamiento
random_search.fit(X_train_tfidf, y_train)

# Obtener el mejor modelo y hacer predicciones
best_svm_model = random_search.best_estimator_
y_pred_svm = best_svm_model.predict(X_test_tfidf)

# Evaluar el modelo optimizado
print("🔹 Mejor parámetro encontrado (Random Search):", random_search.best_params_)
print("🔹 SVM Optimizado - Precisión en Test:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


In [ ]:
# 2. Preparar todos los componentes para guardar
model_components = {
    'model': best_lr_model_random,  # El modelo optimizado
    'random_search': random_search,  # Objeto completo de RandomizedSearchCV
    'vectorizer': vectorizer,  # El vectorizador TF-IDF
    'label_encoder': le,  # El LabelEncoder
    'metadata': {
        'model_type': 'Optimized Logistic Regression (RandomizedSearchCV)',
        'best_parameters': random_search.best_params_,
        'best_score': random_search.best_score_,
        'test_accuracy': accuracy_score(y_test, y_pred_lr_random),
        'classification_report': classification_report(y_test, y_pred_lr_random, output_dict=True),
        'training_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'features': f"{X_train_tfidf.shape[1]} features TF-IDF",
        'classes': list(le.classes_),
        'param_distributions': param_dist,  # Distribución de parámetros usada
        'random_state': 42  # Semilla aleatoria para reproducibilidad
    }
}

# 3. Guardar todo en un archivo .pkl
filename = f"optimized_logistic_regression_random_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.pkl"
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"✅ Modelo de Regresión Logística optimizado (Random Search) guardado exitosamente en:\n{full_path}")
print(f"\n🔹 Mejores parámetros: {random_search.best_params_}")
print(f"🔹 Mejor score (CV): {random_search.best_score_:.4f}")
print(f"🔹 Accuracy (test): {accuracy_score(y_test, y_pred_lr_random):.4f}")
print(f"🔹 Número de iteraciones: {random_search.n_iter}")


## Modelos mas avanzados
### Gradient Boosting


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

# 1. Cargar y preparar datos
# Asumiendo que X e y están definidos
X['sentimiento'] = X['sentimiento'].map({'positivo': 1, 'negativo': -1, 'neutral': 0})
X['sentimiento'] = pd.to_numeric(X['sentimiento'], errors='coerce').fillna(0)

# 2. Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

# 3. Función para convertir tokens a formato compatible
def tokens_to_dict(tokens_series):
    return [dict(zip(tokens, [1]*len(tokens))) for tokens in tokens_series]

# 4. Transformador personalizado para manejar datos dispersos
class DenseTransformer:
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.toarray() if hasattr(X, 'toarray') else X

# 5. Preprocesamiento adaptado
preprocessor = ColumnTransformer(
    transformers=[
        ('text', Pipeline([
            ('tfidf', TfidfVectorizer(
                max_features=5000,
                stop_words='english',
                ngram_range=(1, 2),
                min_df=5,
                max_df=0.7)),
            ('densify', DenseTransformer())
        ]), 'text_limpio'),
        ('num', StandardScaler(), ['helpful_vote', 'length', 'sentimiento']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['verified_purchase']),
        ('tokens', Pipeline([
            ('to_dict', FunctionTransformer(tokens_to_dict)),
            ('vectorize', DictVectorizer()),
            ('densify', DenseTransformer())
        ]), 'tokens')
    ],
    remainder='drop'
)

# 6. Pipeline final
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(
        random_state=42,
        early_stopping=True,
        scoring='accuracy',
        categorical_features=None  # Lo determinaremos después
    ))
])

# 7. Determinar índices de características categóricas
preprocessor.fit(X_train)
n_text_features = preprocessor.named_transformers_['text'].named_steps['tfidf'].get_feature_names_out().shape[0]
n_num_features = 3  # helpful_vote, length, sentimiento
n_cat_features = len(preprocessor.named_transformers_['cat'].get_feature_names_out())

# Las características categóricas son las one-hot encoded (verified_purchase)
categorical_indices = list(range(n_text_features + n_num_features, 
                               n_text_features + n_num_features + n_cat_features))

# 8. Actualizar el pipeline con los índices categóricos
pipeline.set_params(classifier__categorical_features=categorical_indices)

# 9. Parámetros para GridSearchCV (optimizados)
param_grid = {
    'classifier__learning_rate': [0.05, 0.1],
    'classifier__max_iter': [100, 150],
    'classifier__max_depth': [3, 5],
    'classifier__min_samples_leaf': [5, 10],
    'preprocessor__text__tfidf__max_features': [3000, 4000]
}

# 10. Búsqueda de hiperparámetros
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# 11. Entrenamiento
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# 12. Evaluación
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# 2. Preparar los componentes para guardar
model_components = {
    'pipeline': pipeline,  # Tu pipeline completo
    'metadata': {
        'model_type': 'Random Forest Pipeline',
        'features_used': list(X_train.columns),
        'accuracy': accuracy_score(y_test, y_pred),
        'training_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'classes': list(y.unique())
    }
}

# 3. Guardar el modelo
filename = f"random_forest_pipeline_{datetime.datetime.now().strftime('%Y%m%d')}.pkl"
full_path = os.path.join(model_path, filename)

with open(full_path, 'wb') as file:
    pickle.dump(model_components, file)

print(f"Modelo guardado exitosamente en: {full_path}")

### XGBoost

In [ ]:
from sklearn.utils import class_weight
classes = df_reviews_clean['rating'].unique()
weights = class_weight.compute_sample_weight('balanced', df_reviews_clean['rating'])

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import numpy as np

# Paso 1: Re-mapear las etiquetas para que empiecen en 0
y_train_remap = np.array(y_train) - 1  # [1, 2, 3, 4, 5] → [0, 1, 2, 3, 4]
y_test_remap = np.array(y_test) - 1    # Aplicar lo mismo a los datos de test

def objective(trial):
    params = {
        'objective': 'multi:softmax',
        'num_class': 5,  # Número de clases (ahora son [0, 1, 2, 3, 4])
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }
    
    model = XGBClassifier(**params)
    model.fit(X_train_tfidf, y_train_remap)  # Usar y_train_remap
    y_pred = model.predict(X_test_tfidf)
    return f1_score(y_test_remap, y_pred, average='macro')  # Evaluar con y_test_remap

# Paso 2: Ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Paso 3: Mostrar mejores parámetros y resultados
print("Mejores parámetros:", study.best_params)

# (Opcional) Entrenar el modelo final con los mejores parámetros
best_model = XGBClassifier(**study.best_params, objective='multi:softmax', num_class=5)
best_model.fit(X_train_tfidf, y_train_remap)
y_pred = best_model.predict(X_test_tfidf)

# Generar reporte de clasificación
y_pred_original = y_pred + 1  
print(classification_report(y_test, y_pred_original))

### Análisis Comparativo de Métricas

#### **1. Rendimiento General por Modelo**
| Modelo                     | Accuracy | F1-Score (Macro) | F1-Score (Weighted) | Tiempo Estimado |
|----------------------------|----------|-------------------|----------------------|-----------------|
| Naïve Bayes Optimizado      | 0.61     | 0.43              | 0.61                 | ⚡ Más rápido   |
| Regresión Logística (Grid)  | **0.67** | 0.44              | **0.62**             | ⏱️ Medio       |
| SVM Optimizado              | 0.65     | 0.42              | 0.60                 | 🐌 Más lento   |
| **Gradient Boosting**       | 0.65     | 0.37              | 0.58                 | ⏱️ Medio       |
| **XGBoost Optimizado**      | 0.56     | 0.19              | 0.46                 | ⏱️ Medio       |

---

#### **2. Problemas Comunes Identificados**
- **Sesgo hacia la clase mayoritaria (5)**:  
  Todos los modelos (especialmente XGBoost) tienen recall >90% para la clase 5, pero <20% para las clases 2, 3 y 4.  
  - *Ejemplo*: Gradient Boosting tiene **0% recall en clase 2** (no detecta ningún caso).

- **Overfitting en XGBoost**:  
  A pesar de ajustar hiperparámetros, su F1-score macro es muy bajo (0.19), lo que sugiere que está memorizando ruido en los datos.

- **Precisión vs Recall desbalanceados**:  
  - Gradient Boosting tiene alta precisión en clase 1 (82%) pero bajo recall (34%), indicando falsos negativos.  
  - SVM tiene mejor equilibrio (ej: clase 1 con F1=0.55).

---

#### **3. Recomendaciones por Modelo**
##### **Gradient Boosting**
- **Ventajas**:  
  - Buen accuracy (65%) similar a SVM, pero más rápido.  
  - Detecta algo de señal en clases minoritarias (ej: F1=0.31 en clase 3).  
- **Problemas**:  
  - Colapso en clase 2 (F1=0.00).  
  - Matriz de confusión muestra confusión entre clases 1-5 y 3-5.  
- **Acciones**:  
  - Aumentar `min_samples_leaf` (evitar overfitting en hojas).  
  - Usar `class_weight='balanced'` o oversampling (SMOTE).  

##### **XGBoost**
- **Ventajas**:  
  - Arquitectura flexible (permite ajustes finos).  
- **Problemas**:  
  - Pésimo rendimiento en clases 1-4 (F1 <0.10).  
  - Posiblemente necesita más datos o features mejor diseñadas.  
- **Acciones**:  
  - Reducir complejidad (`max_depth=5`, `gamma=0.5`).  
  - Usar `scale_pos_weight` para balancear clases.  

---

#### **4. Comparación con Modelos Previos**
| Métrica               | Mejor Modelo (Regresión Logística) | Gradient Boosting | XGBoost |
|-----------------------|------------------------------------|-------------------|---------|
| **Accuracy**          | **0.67**                           | 0.65              | 0.56    |
| **F1-Score (Macro)**  | **0.44**                           | 0.37              | 0.19    |
| **Recall Clase 2**    | 0.10                               | **0.00**          | 0.01    |
| **Velocidad**         | ⏱️ Medio                          | ⏱️ Medio         | ⏱️ Medio |

---
